In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
# train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [46]:
def read_data(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-4]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    data = np.array(data).transpose(0, 2, 1)
    d_shape = data.shape
    return data.reshape(d_shape[0], 1, d_shape[1], d_shape[2])

def read_label(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-10]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    return np.array(data)

In [51]:
train_data_path = "..//data//train//train_data"
train_label_path = "..//data//train//train_label"
test_data_path = "..//data//test//test_data"
test_label_path = "..//data//test//test_label"

train_data = read_data(train_data_path) # 519
train_label = read_label(train_label_path)
test_data = read_data(test_data_path) # 519
test_label = read_label(test_label_path)

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(519, 1, 6, 1024)
(519, 1024)
(58, 1, 6, 1024)
(58, 1024)


In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1_1 = nn.Conv2d(1, 64, (1,17), padding=(0,8))
        self.conv1_2 = nn.Conv2d(64, 64, (1,17), padding=(0,8))
        
        self.pool = nn.MaxPool2d((1,2), stride=(1,2))

        self.conv2_1 = nn.Conv2d(64, 128, (1,17), padding=(0,8))
        self.conv2_2 = nn.Conv2d(128, 128, (1,17), padding=(0,8))
        
        self.conv3_1 = nn.Conv2d(128, 256, (1,17), padding=(0,8))
        self.conv3_2 = nn.Conv2d(256, 256, (1,17), padding=(0,8))
        self.conv3_3 = nn.Conv2d(256, 256, (1,17), padding=(0,8))
        self.up1 = nn.Upsample(size=(6, 512), mode='bilinear', align_corners=True)
        self.conv3_4 = nn.Conv2d(256, 128, (1,1))
        
        self.conv4_1 = nn.Conv2d(256, 128, (1,17), padding=(0,8))
        self.conv4_2 = nn.Conv2d(128, 128, (1,17), padding=(0,8))
        self.up2 = nn.Upsample(size=(6, 1024), mode='bilinear', align_corners=True)
        self.upconv4_3 = nn.Conv2d(128, 64, (1,1))
        
        self.conv5_1 = nn.Conv2d(128, 64, (1,17), padding=(0,8))
        self.conv5_2 = nn.Conv2d(64, 64, (6, 17), padding=(0,8))
        self.conv5_3 = nn.Conv2d(64, 1, 1)
        
    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        
        x2 = self.pool(x)
        
        x2 = F.relu(self.conv2_1(x2))
        x2 = F.relu(self.conv2_2(x2))
        
        x3 = self.pool(x2)
        
        x3 = F.relu(self.conv3_1(x3))
        x3 = F.relu(self.conv3_2(x3))
        x3 = F.relu(self.conv3_3(x3))
        x3 = self.up1(x3)
        x3 = self.conv3_4(x3)
        
        x4 = torch.cat([x3, x2], dim=1)
        
        x4 = F.relu(self.conv4_1(x4))
        x4 = F.relu(self.conv4_2(x4))
        x4 = self.up2(x4)
        x4 = self.upconv4_3(x4)

        x5 = torch.cat([x4, x], dim=1)
        
        x5 = F.relu(self.conv5_1(x5))
        x5 = F.relu(self.conv5_2(x5))
        x5 = self.conv5_3(x5)
        
        output = torch.sigmoid(x5)
    
        return output
    
model = Net()
if train_on_gpu:
    model.cuda()
print(model)

Net(
  (conv1_1): Conv2d(1, 64, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv1_2): Conv2d(64, 64, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv2_2): Conv2d(128, 128, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv3_1): Conv2d(128, 256, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv3_2): Conv2d(256, 256, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv3_3): Conv2d(256, 256, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (up1): Upsample(size=(6, 512), mode=bilinear)
  (conv3_4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (conv4_1): Conv2d(256, 128, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv4_2): Conv2d(128, 128, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (up2): Upsample(size=(6, 1024), mode=bilinear)
  (upc

In [48]:
# Loss Function and Optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [60]:
test_data = torch.from_numpy(test_data).float()
test_label = torch.from_numpy(test_label).float()

In [53]:
train_data = torch.from_numpy(train_data).float()
train_label = torch.from_numpy(train_label).float()

In [57]:
#Train
import time
t1 = time.time()
epochs = 500
batch_size = 32

for e in range(1, epochs+1):
    running_loss = 0

    for i in range(0, len(train_data), batch_size):
        data = train_data[i:i+batch_size]
        label = train_label[i:i+batch_size]
        if train_on_gpu:
            data = data.cuda()
            label = label.cuda()

        log_ps = model(data)

        loss = criterion(log_ps, label)
        
#         pred = log_ps >= 0.5
#         correct = pred.eq(label).sum() / label.numel()
#         correct = np.squeeze(correct.cpu().numpy())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    if e%10 == 0 or e==1:
        print("Epoch {}, Training loss: {}".format(e, running_loss/len(train_data)))
        
print("Time taken: {}".format(time.time()-t1))

Epoch 1, Training loss: 0.013153555305936662
Epoch 10, Training loss: 0.013033241579068648
Epoch 20, Training loss: 0.012921215768953739
Epoch 30, Training loss: 0.01280708756003536
Epoch 40, Training loss: 0.012701140461859215
Epoch 50, Training loss: 0.012623115871568176
Epoch 60, Training loss: 0.012552769657274662
Epoch 70, Training loss: 0.012488389549227808
Epoch 80, Training loss: 0.012428791536761158
Epoch 90, Training loss: 0.012373087550863365
Epoch 100, Training loss: 0.012320417228921072
Epoch 110, Training loss: 0.012269986244295374
Epoch 120, Training loss: 0.012221519169550181
Epoch 130, Training loss: 0.012175012042527024
Epoch 140, Training loss: 0.012130573650316006
Epoch 150, Training loss: 0.01208808375002102
Epoch 160, Training loss: 0.012046876153504917
Epoch 170, Training loss: 0.012006579178721
Epoch 180, Training loss: 0.011966929815867503
Epoch 190, Training loss: 0.011927075870693993
Epoch 200, Training loss: 0.011889799410093268
Epoch 210, Training loss: 0.0

In [58]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [33]:
asd = torch.from_numpy(np.array([0.5,0.4,0.6]))

In [62]:
test_data = test_data.cuda()
test_label = test_label.cuda()
out = model(test_data)

In [78]:
pred = out >= 0.5
pred = pred.view(58, -1)
correct = pred.eq(test_label)

In [84]:
out.shape
print(pred.shape)
print(test_label.shape)
print(correct.shape)

print(correct.sum())
print(correct.sum()/ (1024*58))

torch.Size([58, 1024])
torch.Size([58, 1024])
torch.Size([58, 1024])
tensor(50226, device='cuda:0')
tensor(0, device='cuda:0')


In [ ]:
out = model(traindata)